# Lesson 2: Advanced Agentic AI

## Grounding Foundation Models & Multi-Agent Committees

In Lesson 1, you learned that LLMs are powerful pattern predictors but have limitations:
- They can't access real-time information
- They don't know about YOUR specific data
- They work in isolation

Today, we'll solve these problems by:
1. **Grounding** LLMs with your own knowledge base
2. **Coordinating** multiple agents to solve complex tasks

# **Guiding Questions:**
1. How can we make LLMs answer questions about information they've never seen?
2. Can multiple AI agents work together better than one?

In [ ]:
import os
from dotenv import load_dotenv

if 'OPENAI_API_KEY' in os.environ:
    print(f" Removing old key: {os.environ['OPENAI_API_KEY'][:15]}...")
    del os.environ['OPENAI_API_KEY']
    print("Cleared from environment")

load_dotenv(override=True)

api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    print(f"\nAPI key loaded (length: {len(api_key)})")
    print(f"   Starts with: {api_key[:7]}...")
    if api_key.startswith('sk-'):
        print("Valid key")
    else:
        print("Invalid key")
else:
    print("No API key in .env")

---

# Part 1: The Hallucination Problem

## What Does "Grounding" Mean?

**Grounding** means connecting an LLM's responses to specific, verifiable sources of information.

### The Problem:
- LLMs only know what was in their training data (cutoff: early 2024 for most models)
- They hallucinate when they don't know something
- They can't access YOUR documents, databases, or private information

### The Solution: RAG (Retrieval-Augmented Generation)
**RAG** = Retrieve relevant information → Augment the prompt → Generate grounded responses

Think of it like an open-book exam vs. a closed-book exam!

## Setup: Import Libraries

Let's set up our environment for building a grounded agent.

In [ ]:
import asyncio

from fairlib.utils.document_processor import DocumentProcessor

from fairlib import (
    settings,
    AbstractTool,
    ManagerPlanner,
    HierarchicalAgentRunner,
    Message,
    OpenAIAdapter,
    ToolRegistry,
    ToolExecutor,
    WorkingMemory,
    LongTermMemory,
    ChromaDBVectorStore,
    ReActPlanner,
    SimpleAgent,
    SentenceTransformerEmbedder,
    SimpleRetriever,
    KnowledgeBaseQueryTool
)

try:
    import chromadb
    CHROMADB_AVAILABLE = True
except ImportError:
    print("Warning: chromadb not installed. Install with: pip install chromadb")
    CHROMADB_AVAILABLE = False

## Experiment 1: The Hallucination Problem

Let's see what happens when we ask an LLM about information it doesn't have.

**Try this**: Google "What time does the Kraemer Family Library at UCCS close on Friday nights?" and see what the actual hours are. Then compare to what the LLM says.

In [ ]:
print("Initializing OpenAI model...")

# this OpenAIAdapter is your interface to OpenAI models [such as gpt-4o-mini]
# fairlib class abstract away the API details to OpenAI, to interface with the llm, call 'llm.invoke' [llm.ainvoke for asynchronus calls]
llm = OpenAIAdapter(
    api_key=api_key, 
    model_name="gpt-4o-mini"
)

print("Model loaded!")
print(f"Using model: gpt-4o-mini")
print("\nModel capabilities:")
capabilities = llm.get_model_capabilities()
for key, value in capabilities.items():
    print(f"  • {key}: {value}")

In [ ]:
question = "What time does the Kraemer Family Library at UCCS close on Friday nights?"

print(f"Question: {question}\n")
print("Asking the LLM (without access to real documents)...\n")

# fairlib 'Message' class [https://github.com/USAFA-AI-Center/fair_llm/blob/main/fairlib/core/message.py]
#       role: define the level at wich the message is coming from:
#             system -> sets up the context for the system (prompt building)
#             user -> a query from the user of the agentic system
#             assistant -> a message from an LLM within the committee of LLMs
#             tool -> the result of executing a tool

# here we are querying the LLM, so user level Message
messages = [Message(role="user", content=question)] 
response = llm.invoke(messages)

print(f"LLM Response:\n{response.content}\n")
print("=" * 80)
print("WARNING: This information might be completely made up!")
print("=" * 80)
print("\n Google 'What time does the Kraemer Family Library at UCCS close on Friday nights?' to verify if this is correct!")
print("\n Did the LLM:")
print("   a) Give the right answer?")
print("   b) Admit it doesn't know?")
print("   c) Make up a plausible-sounding answer?")
print("   d) Could the answer be more accurate?")

### Reflection Question 1

**Did the LLM give you the right answer? Did it admit it doesn't know, or did it make something up?**

This is the hallucination problem: LLMs try to answer even when they don't have the information!

## Building a Knowledge Base with Real Documents

Instead of relying on the LLM's training data, let's create actual documents with UCCS information.

In a real system, you'd have PDFs, Word docs, or web pages. For this demo, we'll create text files.

In [ ]:
# Create a directory for our knowledge base documents
kb_directory = "./uccs_knowledge_base"
os.makedirs(kb_directory, exist_ok=True)

# Document 1: Library Hours
library_hours_doc = """Kraemer Family Library Hours (UCCS)

Fall/Spring Semester Regular Hours:
- Monday - Thursday: 7:30 AM - 11:00 PM
- Friday: 7:30 AM - 6:00 PM
- Saturday: 10:00 AM - 6:00 PM
- Sunday: 12:00 PM - 11:00 PM

Finals Week Extended Hours:
- Sunday - Thursday: 7:30 AM - 2:00 AM
- Friday: 7:30 AM - 6:00 PM
- Saturday: 10:00 AM - 6:00 PM

Summer and Break Hours:
- Monday - Friday: 8:00 AM - 5:00 PM
- Closed on weekends

Note: Hours may vary during holidays. Always check library.uccs.edu for the most current information.
"""

# Document 2: Degree Requirements
degree_requirements_doc = """UCCS Computer Science B.S. Degree Requirements

Total Credit Hours Required: 120

Core Computer Science Courses: 48 credit hours
- CS 1030, 1050, 1150, 1200, 2060, 2100, 2400
- CS 3100, 3150, 3300, 3400, 3500, 4200
- CS 4310 (Senior Design I) and CS 4320 (Senior Design II)

Mathematics Requirements: 16 credit hours
- MATH 1310, 1320, 2130, 2420

General Education: 35 credit hours
- UCCS Core and Compass Curriculum requirements

Electives and Free Choice: 21 credit hours

Additional Requirements:
- Minimum 2.0 GPA in major courses
- At least 30 credit hours must be upper-division (3000-4000 level)
- Senior design capstone project required
- Complete at least 45 credit hours at UCCS
"""

# Document 3: Campus Resources
campus_resources_doc = """UCCS Student Resources and Services

Writing Center:
- Location: Columbine Hall, Room 108
- Services: Free writing tutoring for all UCCS students
- Walk-in hours: Monday-Friday, 9:00 AM - 5:00 PM
- Online appointments available via Zoom
- Website: writingcenter.uccs.edu

Wellness Center:
- Location: University Center, Second Floor
- Services: Medical care, counseling, health education
- Phone: 719-255-4444
- Hours: Monday-Friday, 8:00 AM - 5:00 PM
- Crisis support available 24/7

Career Center:
- Location: Cragmor Hall, Room 111
- Services: Resume reviews, interview preparation, job search assistance
- Handshake platform for job/internship postings
- Career fairs held each semester (Fall and Spring)
- One-on-one advising appointments available

Math Learning Center:
- Location: Engineering Building, Room 105
- Free tutoring for math courses up to Calculus II
- Walk-in hours: Monday-Thursday, 10:00 AM - 6:00 PM

Computer Science Tutoring Lab:
- Location: Engineering Building, Room 207
- Free tutoring for CS 1030, 1050, 1150, 1200, 2060
- Hours: Monday-Friday, 12:00 PM - 5:00 PM
"""

with open(os.path.join(kb_directory, "library_hours.txt"), "w") as f:
    f.write(library_hours_doc)

with open(os.path.join(kb_directory, "degree_requirements.txt"), "w") as f:
    f.write(degree_requirements_doc)

with open(os.path.join(kb_directory, "campus_resources.txt"), "w") as f:
    f.write(campus_resources_doc)

print("Knowledge base documents created!")
print(f"Saved to: {kb_directory}")
print(f"Documents: library_hours.txt, degree_requirements.txt, campus_resources.txt")

## Processing Documents with FAIR_LLM

Now we'll use FAIR_LLM's `DocumentProcessor` to load and chunk these documents.

This is the same process you'd use for PDFs, Word docs, PowerPoints, etc.!

In [ ]:
# fairlib 'DocumentProcessor' class [https://github.com/USAFA-AI-Center/fair_llm/blob/main/fairlib/utils/document_processor.py]
#       This class handles injecting documents of various types into the VectorStore we will define in a few cells
#       Accepted Document Types:
#                 - .pdf
#                 - .pptx
#                 - .xlsx
#                 - .txt
#                 - .csv
#                 - .sql
doc_processor = DocumentProcessor(config={
    "files_directory": kb_directory,
    "max_chunk_chars": 1000,
    "supported_extensions": {".txt", ".pdf", ".docx"}
})

print("Document processor initialized!")

In [ ]:
# fairlib 'Document' class [https://github.com/USAFA-AI-Center/fair_llm/blob/main/fairlib/core/types.py]
#       A very basic class with two member variables
#           - self.page_content: The string content of the document
#           - self.metadata: Dict -> Data about page_content (execute cell and check below)

# this will be a list of [Document()] objects
documents = doc_processor.load_documents_from_folder()

print(f"\nLoaded {len(documents)} document chunks")
print(f"\nExample chunk:")
print(f"Content: {documents[0].page_content[:200]}...")
print(f"\nMetadata: {documents[0].metadata}")

print("""\n\nThis is a perfect example of Fairlib abstracting the difficulty of creating agentic pipelines, you dont need 
to implement the code to process a pdf, just choose the correct pdfs to ground your agents and leverage our classes.""")

## Building the RAG Architecture

Now we'll build the proper RAG pipeline with the core FAIR_LLM components:

1. **Embedder**: Converts text into numerical vectors
2. **Vector Store**: Stores embeddings and enables similarity search (semantic)  
3. **Long-Term Memory**: Wraps the vector store
4. **Retriever**: Queries the vector store to find relevant documents

**How it works:**
1. Documents are converted into vector embeddings (numerical representations)
2. Embeddings are stored in a vector database (ChromaDB)
3. When you query, the query is converted to a vector
4. Vector store finds the most similar document vectors (semantic search)
5. Retriever returns the most relevant chunks

In [ ]:
# Check if ChromaDB is available
if not CHROMADB_AVAILABLE:
    print("ERROR: ChromaDB is required for this section.")
    print("Install it with: pip install chromadb")
    print("\nSkipping RAG setup...")
else:
    print("ChromaDB available")

In [ ]:
# Step 1: Create the embedder
print("Step 1: Creating embedder...")

# the fairlib 'SentenceTransformerEmbedder' class [https://github.com/USAFA-AI-Center/fair_llm/blob/main/fairlib/modules/memory/embedder.py#L55C7-L55C34]
#      to create an instance of this class, pass an embedder model name that is compliant with:
#           'self.model = SentenceTransformer(model_name)'
#      ScentenceTransformer() is not a fairlib class, from the 'sentence_transformers' library
#      If you leave the model_name field blank, fairlib will default to -> sentence-transformers/multi-qa-mpnet-base-dot-v1 
embedder = SentenceTransformerEmbedder(
    model_name="sentence-transformers/all-MiniLM-L6-v2"  # Fast, efficient model
)


print("Embedder created")

In [ ]:
# Step 2 & 3: Create vector store and add documents
print("\nStep 2 & 3: Creating vector store and adding documents...")

# Create a fresh ChromaDB client
chroma_client = chromadb.Client()

# Delete collection if it exists (prevents duplicate IDs on reruns)
try:
    chroma_client.delete_collection(name="uccs_knowledge_base")
    print("  (Cleared existing collection)")
except:
    pass

# the fairlib 'ChromaDBVectorStore' class [https://github.com/USAFA-AI-Center/fair_llm/blob/main/fairlib/modules/memory/vector_store.py#L82]
#        this class leverages the 'chromadb' python class, documentation: https://www.trychroma.com/
vector_store = ChromaDBVectorStore(
    embedder=embedder,
    client=chroma_client,
    collection_name="uccs_knowledge_base"
)
print("Vector store created")

# Extract text content from Document objects
document_texts = [doc.page_content for doc in documents]

# Remove duplicate documents (ChromaDB uses hash(doc) as ID internally)
unique_texts = []
seen = set()
for text in document_texts:
    if text not in seen:
        seen.add(text)
        unique_texts.append(text)

print(f"  Total chunks: {len(document_texts)}, Unique chunks: {len(unique_texts)}")

# Add to vector store
vector_store.add_documents(unique_texts)

print(f"Added {len(unique_texts)} document chunks to vector store")

In [ ]:
print("\nStep 4: Creating long-term memory...")

# the fairlib 'LongTermMemory' class [https://github.com/USAFA-AI-Center/fair_llm/blob/main/fairlib/modules/memory/base.py#L35]
#        - use this class for long term data storage (when performing RAG)
#        - this classes counterpart 'WorkingMemory' should be used for things like your conversation history of the current chat
long_term_memory = LongTermMemory(vector_store)
print("Long-term memory created")

In [ ]:
# the fairlib 'SimpleRetriever' class [https://github.com/USAFA-AI-Center/fair_llm/blob/main/fairlib/modules/memory/retriever.py#L11]
#       - This class should be seem as your interface to the vector_store
#       - There are only two methods defined in this class:
#                retrieve(self, query: str, top_k: int = 5) -> synchronosly search the vector store
#                aretrieve(self, query: str, top_k: int = 5) -> asynchronosly search the vector store
#       - Both of these methods return 'List[Document]'
retriever = SimpleRetriever(vector_store)
print("✓ Retriever created and ready!")
print("\n" + "="*60)
print("RAG PIPELINE COMPLETE")
print("="*60)

### Test the Retriever Directly

Let's see how semantic search works!

In [ ]:
# Test retrieval
test_query = "library hours friday"
results = retriever.retrieve(query=test_query, top_k=2)

print(f"Query: '{test_query}'")
print(f"\nTop {len(results)} most relevant chunks:\n")

for i, result in enumerate(results, 1):
    print(f"--- Result {i} ---")
    # Document object
    if hasattr(result, 'metadata'):
        print(f"Source: {result.metadata.get('source', 'unknown')}")
    print(f"Content: {result.page_content[:400] if hasattr(result, 'page_content') else str(result)[:400]}...\n")

## Building a Grounded Agent with RAG

Now we'll create an agent that uses the `KnowledgeBaseQueryTool` to ground its responses!

In [ ]:
# Create a separate LLM instance for the agent
agent_llm = llm

# Create the knowledge base query tool
# the fairlib 'KnowledgeBaseQueryTool' [https://github.com/USAFA-AI-Center/fair_llm/blob/main/fairlib/modules/action/tools/knowledge_tool.py#L28]
#        this class is responsible for calling the retriever class defined in the previous cells
knowledge_tool = KnowledgeBaseQueryTool(retriever=retriever)

# Register the tool
tool_registry = ToolRegistry()
tool_registry.register_tool(knowledge_tool)

# Create executor, memory, and planner
executor = ToolExecutor(tool_registry)
memory = WorkingMemory()
planner = ReActPlanner(agent_llm, tool_registry)

# Assemble the agent
grounded_agent = SimpleAgent(
    llm=agent_llm,
    planner=planner,
    tool_executor=executor,
    memory=memory,
    max_steps=5
)

grounded_agent.role_description = (
    "You are a UCCS student assistant. "
    "Your job is to answer student questions about UCCS accurately. "
    "ALWAYS search the knowledge base before answering - never make up information. "
    "Use the course_knowledge_query tool to find relevant information, then base your answer on those documents. "
    "Always answer based off of the documents you find from the course_knowledge_query tool."
    "If the information isn't in the knowledge base, say so and suggest where students can find more information."
    "The ONLY tool you have access to is the course_knowledge_query tool. Do NOT attempt to use other tools."
)

print("Grounded agent created successfully!")
print("Agent has access to UCCS knowledge base via RAG")

## Testing the Grounded Agent

Let's ask the same question - but now the agent has access to real documents!

In [ ]:
async def test_grounded_agent(question):
    print(f"Question: {question}")
    print("\nAgent thinking...\n")
    
    response = await grounded_agent.arun(question)
    
    print(f"\n{'='*60}")
    print("Agent Response:")
    print(f"{'='*60}")
    print(response)
    print(f"{'='*60}\n")
    return response

In [ ]:
# Test 1: Question about library hours (the one that was hallucinated before!)
await test_grounded_agent("What time does the Kraemer Family Library at UCCS close on Friday nights?")

In [ ]:
# Test 2: Question about degree requirements
await test_grounded_agent("How many credit hours are required to graduate with a Computer Science degree from UCCS?")

In [ ]:
# Test 3: Question about campus resources
await test_grounded_agent("Where can I get help with my resume at UCCS?")

In [ ]:
# Test 4: Question about something NOT in the documents
await test_grounded_agent("What is the tuition cost for in-state students at UCCS?")

## Breaking the Grounded Agent

Let's ask some malformed or out of scope questions!

In [ ]:
print("\n--- Test 1: Out of scope ---")
out_of_scope_question = "What's the weather forecast for tomorrow?"

await test_grounded_agent(out_of_scope_question)

In [ ]:
print("\n--- Test 2: Multi-document reasoning ---")
complex_question = "If I'm a CS major, what building is my tutoring center in and when is it open?"

await test_grounded_agent(complex_question)

In [ ]:
print("\n--- Test 3: Typos ---")
typo_question = "librayr hourz fryday"

await test_grounded_agent(typo_question)

### Reflection Question 2

**What's different between the pure LLM and the grounded agent?**

Notice:
- The grounded agent **searches documents first** before answering
- Answers are **based on actual source text** (you can verify!)
- When info isn't available, it **admits it and suggests alternatives** (no hallucination!)
- You can **trace where the information came from**

This is **Retrieval-Augmented Generation (RAG)** in action!

## Understanding the RAG Pipeline

### Without Grounding (Pure LLM):
```
User: "What's the library closing time on Friday?"
  ↓
LLM: [Predicts based on general patterns]
  ↓
LLM: "Most libraries close around 10 PM on Fridays..." (HALLUCINATION!)
```

### With Grounding (RAG Agent):
```
User: "What's the library closing time on Friday?"
  ↓
Agent: [Uses course_knowledge_query tool]
  ↓
KnowledgeBaseQueryTool: [Calls retriever]
  ↓
SimpleRetriever: [Queries vector store]
  ↓
ChromaDBVectorStore: [Semantic search via embeddings]
  ↓
Returns: "Friday: 7:30 AM - 6:00 PM" from library_hours.txt
  ↓
Agent: [Generates answer based on retrieved text]
  ↓
Agent: "According to the library hours, Kraemer Library closes at 6:00 PM on Fridays."
```

### The RAG Architecture Layers:
```
Agent (Decision Making)
   |
KnowledgeBaseQueryTool (Interface)
   |
SimpleRetriever (Query Logic)
   |
LongTermMemory (Abstraction)
   |
ChromaDBVectorStore (Storage + Search)
   |
SentenceTransformerEmbedder (Vectorization)
```

Each layer has a specific responsibility:
- **Embedder**: Converts text to vectors
- **Vector Store**: Stores and searches embeddings
- **Long-Term Memory**: Provides memory abstraction
- **Retriever**: Implements retrieval logic
- **Tool**: Formats results for the agent
- **Agent**: Makes decisions about when to use the tool

**Key Benefits:**
- ✅ Factual and verifiable
- ✅ Based on YOUR documents
- ✅ Up-to-date (just update the docs!)
- ✅ Transparent (you can see the sources)
- ✅ Modular (swap components as needed)

---

# Part 2: Committees of AI Agents

## Why Use Multiple Agents?

Just like human teams, different agents can have different:
- **Roles** (specialist vs. generalist)
- **Tools** (some agents have access to certain resources)
- **Expertise** (some are better at specific tasks)

### Real-World Examples:
- **Code Review Committee**: One agent writes code, another reviews for bugs, another checks style
- **Essay Grading Team**: One checks grammar, another evaluates argument quality, another verifies facts
- **Research Team**: One searches papers, another summarizes, another synthesizes findings

## Example Task: Essay Evaluation

Let's build a committee to grade essays with three specialized agents:
1. **Grammar Agent**: Checks spelling, grammar, and clarity
2. **Content Agent**: Evaluates argument quality and evidence
3. **Coordinator Agent**: Synthesizes feedback and assigns final grade

## Sample Essay for Testing

In [ ]:
sample_essay = """The Impact of Artificial Intelligence on Education

Artificial intelligence is revolutionizing education in many ways. AI can personalize learning 
by adapting to each students pace and style. For example, intelligent tutoring systems can 
identify where a student struggles and provide targeted practice.

However, their are concerns about AI in education. Some worry that students might become to 
dependent on AI tools and not develop critical thinking skills. Others point out that AI 
systems can perpetuate biases if there trained on biased data.

Despite these challenges, the benefits outweigh the risks. AI can help teachers by automating 
administrative tasks, allowing them to focus on actual teaching. It can also make education 
more accessible to students in remote areas who otherwise wouldn't have access to quality 
instruction.

In conclusion, AI has the potential to transform education for the better, but we must 
implement it thoughtfully and address the ethical concerns. The future of education will 
likely involve a partnership between human teachers and AI systems, combining the best of both.
"""

print("Sample essay created.")
print(f"Length: {len(sample_essay.split())} words")

## Building Specialized Agent Tools

First, let's create tools that represent specialized capabilities.

In [ ]:
# THESE CELLS ARE EXAPLES OF HOW YOU COULD IMPLEMENT YOUR OWN TOOLS IN YOUR OWN AGENTIC SYSTEM. VERY GOOD REFERENCE.

# the fairlib 'AbsractTool' class [https://github.com/USAFA-AI-Center/fair_llm/blob/main/fairlib/core/interfaces/tools.py]
#       - This class is the contract that you must adhere to to define your own custom tools
#       - To define a tool that adheres to this contract it must have:
#             name: str -> a name that the LLM will use to call this tool
#             description: str -> to describe the tool to the LLM
#             use(tool_input: str): function -> use the tool
#
#
# IMPORTANT NOTE: if you adhear to this contract, fairlib will handle the logic behind the LLM calling your tools for you!
class GrammarCheckTool(AbstractTool):
    """Tool for checking grammar and writing quality"""
    
    name = "check_grammar"
    description = (
        "Analyzes text for grammar, spelling, and clarity issues. "
        "Input: essay text. Returns: detailed grammar feedback."
    )
    
    def use(self, tool_input: str) -> str:
        """Simulate grammar checking"""
        issues = []
        
        # Simple checks (real systems use NLP libraries like LanguageTool, you would leverage those systems)
        if "their are" in tool_input.lower():
            issues.append("- Found 'their are' - should be 'there are' (wrong homophone)")
        if "to dependent" in tool_input.lower():
            issues.append("- Found 'to dependent' - should be 'too dependent' (wrong homophone)")
        if "students pace" in tool_input.lower():
            issues.append("- Found 'students pace' - missing apostrophe: 'student's pace'")
        if "there trained" in tool_input.lower():
            issues.append("- Found 'there trained' - should be 'they're trained' (contraction needed)")
        
        if issues:
            return "Grammar Issues Found:\n" + "\n".join(issues) + "\n\nOverall: Multiple homophone and apostrophe errors detected."
        else:
            return "No major grammar issues detected. Writing is clear and correct."


class ContentAnalysisTool(AbstractTool):
    """Tool for analyzing argument quality and evidence"""
    
    name = "analyze_content"
    description = (
        "Evaluates the quality of arguments and evidence in an essay. "
        "Input: essay text. Returns: content quality assessment."
    )
    
    def use(self, tool_input: str) -> str:
        """Simulate content analysis"""
        text_lower = tool_input.lower()
        
        # Count evidence markers
        evidence_markers = ["for example", "research shows", "studies indicate", "data suggests"]
        evidence_count = sum(1 for marker in evidence_markers if marker in text_lower)
        
        # Check structure
        has_intro = "introduction" in text_lower or tool_input.strip().startswith(("Artificial", "The", "In"))
        has_conclusion = "conclusion" in text_lower or "in summary" in text_lower
        
        # Check for counterarguments
        has_counterargument = any(word in text_lower for word in ["however", "although", "despite", "concern"])
        
        feedback = "Content Analysis Results:\n\n"
        feedback += f"Structure:\n"
        feedback += f"  - Clear introduction: {'Yes' if has_intro else 'No'}\n"
        feedback += f"  - Clear conclusion: {'Yes' if has_conclusion else 'No'}\n\n"
        feedback += f"Argumentation:\n"
        feedback += f"  - Evidence examples provided: {evidence_count}\n"
        feedback += f"  - Addresses counterarguments: {'Yes' if has_counterargument else 'No'}\n\n"
        
        if evidence_count < 2:
            feedback += "Suggestion: Add more specific examples and evidence to support claims. "
            feedback += "Consider citing research studies or real-world data.\n"
        if has_counterargument:
            feedback += "Strength: Essay acknowledges opposing views, showing critical thinking.\n"
        
        return feedback


# Create the tools
grammar_tool = GrammarCheckTool()
content_tool = ContentAnalysisTool()

print("Specialized tools created!")

## Creating Specialized Agents

Now we'll create three agents with different roles and tools.

In [ ]:
def create_worker_agent(llm, tools, role_description):
    """
    Factory function to create specialized worker agents.
    Each worker gets its own tool registry, planner, executor, and memory.
    """
    tool_registry = ToolRegistry()
    for tool in tools:
        tool_registry.register_tool(tool)
    
    planner = ReActPlanner(llm, tool_registry)
    executor = ToolExecutor(tool_registry)
    memory = WorkingMemory()
    
    # Create a stateless agent (important for workers!)
    agent = SimpleAgent(llm, planner, executor, memory, stateless=True)
    
    # Set role description
    agent.role_description = role_description
    
    return agent

print("Worker agent factory function created")

In [ ]:
# Worker 1: Grammar Specialist
print("Creating Grammar Specialist worker...")

grammar_llm = OpenAIAdapter(api_key=api_key, model_name="gpt-4o-mini")

grammar_agent = create_worker_agent(
    llm=grammar_llm,
    tools=[grammar_tool],
    role_description=(
        "A Grammar Specialist and writing instructor. "
        "Your job is to evaluate grammar, spelling, and writing clarity. "
        "Use the check_grammar tool to analyze text, then provide specific feedback. "
        "Rate grammar quality on a scale of 1-10 and explain your rating."
    )
)

print("Grammar Specialist worker created")

In [ ]:
# Worker 2: Content Specialist
print("Creating Content Specialist worker...")

content_llm = OpenAIAdapter(api_key=api_key, model_name="gpt-4o-mini")

content_agent = create_worker_agent(
    llm=content_llm,
    tools=[content_tool],
    role_description=(
        "A Content Specialist and critical thinking instructor. "
        "Your job is to evaluate argument quality, evidence, and logical structure. "
        "Use the analyze_content tool to assess text. "
        "Rate content quality on a scale of 1-10 and explain your rating."
    )
)

print("Content Specialist worker created")

In [ ]:
# Organize workers in a dictionary for the manager
workers = {
    "GrammarSpecialist": grammar_agent,
    "ContentSpecialist": content_agent
}

print(f"Team assembled with {len(workers)} specialist workers")
print(f"  Workers: {', '.join(workers.keys())}")

In [ ]:
# Create the Manager Agent (coordinator)
print("\nCreating Manager Agent (Lead Evaluator)...")

manager_llm = OpenAIAdapter(api_key=api_key, model_name="gpt-4o-mini")

# Create memory for the manager
manager_memory = WorkingMemory()

# the fairlib 'ManagerPlanner' class [https://github.com/USAFA-AI-Center/fair_llm/blob/main/fairlib/modules/agent/multi_agent_runner.py#L98]
#        this class is similar to the ReAct planner [https://github.com/USAFA-AI-Center/fair_llm/blob/main/fairlib/modules/planning/react_planner.py#L244]
#        the key difference of this class, is the prompting done to orient the manager to properly facilitate communications between the agents!
manager_planner = ManagerPlanner(manager_llm, workers)

# Create manager agent (note: executor is None - manager only delegates!)
manager_agent = SimpleAgent(
    llm=manager_llm,
    planner=manager_planner,
    tool_executor=None,  # Manager doesn't execute tools directly
    memory=manager_memory
)

# Set manager's role
manager_agent.role_description = (
    "You are the Lead Evaluator managing an essay grading team. "
    "Delegate tasks to GrammarSpecialist and ContentSpecialist workers. "
    "Once you have both reports, synthesize them into a final grade and feedback."
)

print("Manager Agent created")

## Running the Committee

Now let's coordinate the agents to evaluate our essay!

In [ ]:
# Initialize the Hierarchical Agent Runner
print("\nInitializing Hierarchical Agent Runner...")

# the fairlib 'HierarchicalAgentRunner' class [https://github.com/USAFA-AI-Center/fair_llm/blob/main/fairlib/modules/agent/multi_agent_runner.py#L165]
#      this class contains the custom logic to facilitate communication between multiple ai agents
#      another example of the framework abstracting the difficulty of interfacing with agentic systems
#      if you can assemble all of the agents in your committee properly, fairlib will handle their ability to communicate (assuming you prompted them correctly)
essay_committee = HierarchicalAgentRunner(
    manager_agent=manager_agent,
    workers=workers,
    max_steps=10  # Maximum coordination steps
)

print("Essay Evaluation Committee assembled!")
print("\nCommittee Structure:")
print("  Manager: Lead Evaluator")
print("  ├─ GrammarSpecialist")
print("  └─ ContentSpecialist")

In [ ]:
async def evaluate_essay_with_committee(essay_text):
    """
    Coordinate multiple agents to evaluate an essay using hierarchical delegation.
    """
    
    print("="*70)
    print(" " * 20 + "ESSAY EVALUATION COMMITTEE")
    print("="*70)
    
    # Create the evaluation request for the manager
    # THIS IS THE PROMPT ENGINEERING I HAVE BEEN TALKING ABOUT
    evaluation_request = f"""
Please evaluate this essay comprehensively:

ESSAY:
{essay_text}

INSTRUCTIONS:
1. First, delegate to GrammarSpecialist to evaluate grammar and writing quality
2. Then, delegate to ContentSpecialist to evaluate content and argumentation
3. Finally, synthesize both reports into:
   - Overall Grade (A, B, C, D, or F)
   - Summary of Strengths (2-3 points)
   - Key Areas for Improvement (top 3)
   - One Actionable Next Step for the student
"""
    
    print("\n[MANAGER COORDINATING EVALUATION...]\n")
    
    # Run the hierarchical agent team
    # THE MAIN FUNCTION DRIVING THE LOOP [https://github.com/USAFA-AI-Center/fair_llm/blob/main/fairlib/modules/agent/multi_agent_runner.py#L172]
    final_evaluation = await essay_committee.arun(evaluation_request)
    
    print("\n" + "="*70)
    print(" " * 25 + "FINAL EVALUATION")
    print("="*70)
    print(final_evaluation)
    print("\n" + "="*70)
    
    return final_evaluation

In [ ]:
# Run the committee evaluation!
await evaluate_essay_with_committee(sample_essay)

### Reflection Question 3

**What advantages does the hierarchical multi-agent approach have?**

Think about:
- **Manager-Worker Pattern**: Clear delegation and coordination
- **Specialization**: Each worker focuses on one task with dedicated tools
- **Stateless Workers**: Workers don't maintain conversation history (more efficient)
- **Automatic Orchestration**: `HierarchicalAgentRunner` manages the workflow
- **Synthesis by Manager**: Manager combines results into coherent final answer
- **Scalability**: Easy to add more specialists to the team

**How is this different from our earlier sequential approach?**
- Sequential: We manually called each agent and passed results around
- Hierarchical: Manager decides when and how to delegate automatically
- Hierarchical: Manager synthesizes results intelligently based on observations

## Understanding Hierarchical Multi-Agent Systems

### The Architecture:
```
User Query
    ↓
Manager Agent (ManagerPlanner)
    ├─→ Delegates to GrammarSpecialist
    │   └─→ Returns grammar report
    ├─→ Delegates to ContentSpecialist
    │   └─→ Returns content report
    └─→ Synthesizes final evaluation
    ↓
Final Answer to User
```

### Key Components:

**ManagerPlanner:**
- Special planner that understands delegation
- Has two "tools": `delegate` and `final_answer`
- Decides which worker to use based on the task

**Worker Agents:**
- Stateless (don't maintain conversation history)
- Each has specific tools and expertise
- Return results to manager via observations

**HierarchicalAgentRunner:**
- Orchestrates the entire workflow
- Routes delegations to correct workers
- Passes worker results back to manager as observations
- Manages the coordination loop

### Why This Pattern Works:

1. **Clear Separation**: Manager thinks strategically, workers execute tactically
2. **Flexible Coordination**: Manager can adapt based on worker results
3. **Production-Ready**: Same pattern used in complex enterprise systems
4. **Explainable**: You can see the entire decision chain

## Try It Yourself!

Test the committee on your own essay or writing sample:

In [ ]:
your_essay = """
Paste your essay here!
"""

# Uncomment to test:
# await evaluate_essay_with_committee(your_essay)

---

## Key Insights: Multi-Agent Systems

### Why Committees Beat Single Agents:

**Specialization**:
- Each agent masters one skill
- Like human experts, focused agents perform better
- Reduces cognitive load per agent

**Scalability**:
- Add new specialists as needed
- Agents can work in parallel for speed
- Modular architecture is easier to maintain

**Reliability**:
- Multiple perspectives catch more issues
- One agent's weakness is another's strength
- Cross-validation reduces errors

**Transparency**:
- See each agent's reasoning
- Easier to debug and improve
- Clear audit trail

### When to Use Committees:
- ✅ Complex tasks requiring multiple skills
- ✅ Quality-critical applications (grading, review, analysis)
- ✅ Tasks that benefit from different perspectives
- ✅ When you need explainability and transparency

### When NOT to Use Committees:
- ❌ Simple queries (overkill and slower)
- ❌ Extremely time-sensitive tasks (coordination adds latency)
- ❌ Tasks requiring deep context sharing (single agent is easier)

---

## Combining Grounding + Committees

The most powerful systems combine BOTH techniques:

```
Example: Fact-Checked Essay Grader

Grammar Agent (specialist)
     ↓
Content Agent (specialist)
     ↓
Fact-Checker Agent (grounded on knowledge base via RAG)
     ↓
Coordinator (synthesizes all feedback)
     ↓
Final Report with verified facts
```

Each agent is:
- **Grounded** (has access to relevant knowledge via RAG)
- **Specialized** (focuses on one task)
- **Coordinated** (works with other agents)

### Real-World Application Ideas:
1. **Research Assistant**: Search agent (RAG on papers) → Summarizer → Synthesizer
2. **Code Review**: Linter → Security Checker (grounded on CVE database) → Style Reviewer
3. **Customer Support**: Intent Classifier → Knowledge Base Agent (RAG) → Response Generator

## Final Reflection

### 1. How does grounding solve the hallucination problem?
### 2. What are the tradeoffs between single agents and committees?
### 3. Can you think of a real-world task that would benefit from BOTH grounding AND multiple agents?
### 4. How might you apply these techniques to your final project?

## Congratulations!

You've learned advanced agentic AI techniques:

### Grounding Foundation Models:
- ✓ Why LLMs hallucinate
- ✓ How RAG works (Retrieve → Augment → Generate)
- ✓ Using DocumentProcessor to load documents
- ✓ Creating SimpleRetriever for semantic search
- ✓ Building agents with KnowledgeBaseQueryTool
- ✓ Verifiable, source-backed responses

### Multi-Agent Committees:
- ✓ Agent specialization and roles
- ✓ Coordinating multiple agents
- ✓ Sequential and parallel workflows
- ✓ Synthesizing diverse feedback
- ✓ When to use committees vs. single agents

### What's Next?
- Experiment with different committee structures
- Build agents grounded on YOUR data (PDFs, docs, websites)
- Combine techniques for your final projects
- Explore advanced patterns (debate, voting, hierarchies)
- Build production-ready multi-agent systems

---

## Additional Challenges (Optional)

If you want to explore further, try these:

### Challenge 1: Expand the Knowledge Base
Add more UCCS documents (parking info, dining services, athletics) and test the grounded agent.

### Challenge 2: Add a Third Specialist
Create a "Citation Checker" or "Fact Verifier" agent that uses RAG to verify claims in essays.

### Challenge 3: Parallel Evaluation
Modify the committee to run grammar and content agents in parallel using `asyncio.gather()`:
```python
grammar_task, content_task = await asyncio.gather(
    grammar_agent.arun(prompt),
    content_agent.arun(prompt)
)
```

### Challenge 4: Grounded Fact-Checker
Add a fourth agent that uses the knowledge base to verify factual claims in essays.

### Challenge 5: Your Own Multi-Agent System
Design a committee for a task relevant to your final project. Consider:
- What specialists are needed?
- Which agents need RAG access?
- Sequential or parallel coordination?
- How to synthesize results?